In [1]:
import pandas as pds

from sqlalchemy import create_engine

# Import Dependencies
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, MetaData, Table

# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder

import tensorflow as tf


In [2]:
# Create an engine for the chinook.sqlite database
# engine = create_engine("austincrime.cszdqowqamd3.us-east-1.rds.amazonaws.com", echo=False)
import psycopg2 as pg
import pandas.io.sql as psql
connection = pg.connect("host=austincrime.cszdqowqamd3.us-east-1.rds.amazonaws.com dbname=austincrime user=postgres password=T1m3T0Party")
# dataframe = psql.read_sql('SELECT * FROM product_product', connection)
# product_category = psql.read_sql_query('select * from product_category', connection)

In [3]:
crime_df = psql.read_sql('SELECT * FROM austincrime', connection)

In [4]:
crime_df.head()

,ID,CouncilDistrict,GOHighestOffDesc,NIBRSUCROffDesc,ReportDate,Location,Clearance,ClearanceDate,APDDistrict,LocationZip,CensusTract,XCoordinate,YCoordinate
0,201510782,4.0,AGG ROBBERY/DEADLY WEAPON,Robbery,2015-01-01,9001 N IH 35 SVRD NB ...,N,2015-01-28,E,78753.0,18.13,3130483.0,10102366.0
1,201511231,4.0,ROBBERY BY ASSAULT,Robbery,2015-01-01,919 E KOENIG LN SVRD EB ...,N,2015-01-13,I,78751.0,21.05,3124730.0,10090296.0
2,201511736,1.0,BURGLARY OF RESIDENCE,Burglary,2015-01-01,12151 N IH 35 SVRD NB ...,N,2015-01-13,E,78753.0,18.35,3135985.0,10117220.0
3,201511433,4.0,BURGLARY OF RESIDENCE,Burglary,2015-01-01,1044 NORWOOD PARK BLVD ...,N,2015-01-05,I,78753.0,18.13,3129896.0,10096032.0
4,201511936,2.0,BURGLARY OF RESIDENCE,Burglary,2015-01-01,2413 BITTER CREEK DR ...,N,2015-01-07,F,78744.0,24.27,3110455.0,10039340.0


In [5]:
# Generate our categorical variable list
crime_cat = crime_df.dtypes[crime_df.dtypes == "object"].index.tolist()

In [6]:
print(crime_cat)

['ID', 'GOHighestOffDesc', 'NIBRSUCROffDesc', 'ReportDate', 'Location', 'Clearance', 'ClearanceDate', 'APDDistrict']


In [7]:
# Check the number of unique values in each column
crime_df[crime_cat].nunique()

ID                  148278
GOHighestOffDesc       105
NIBRSUCROffDesc          7
ReportDate            1461
Location             51742
Clearance                3
ClearanceDate         1562
APDDistrict             12
dtype: int64

In [8]:
cleaned_crime_df = crime_df.drop(['ID','GOHighestOffDesc','Location', 'ClearanceDate', 'Clearance'],axis = 1)
cleaned_crime_df.head()

,CouncilDistrict,NIBRSUCROffDesc,ReportDate,APDDistrict,LocationZip,CensusTract,XCoordinate,YCoordinate
0,4.0,Robbery,2015-01-01,E,78753.0,18.13,3130483.0,10102366.0
1,4.0,Robbery,2015-01-01,I,78751.0,21.05,3124730.0,10090296.0
2,1.0,Burglary,2015-01-01,E,78753.0,18.35,3135985.0,10117220.0
3,4.0,Burglary,2015-01-01,I,78753.0,18.13,3129896.0,10096032.0
4,2.0,Burglary,2015-01-01,F,78744.0,24.27,3110455.0,10039340.0


In [9]:
 cleaned_crime_df['LocationZip'] = cleaned_crime_df['LocationZip'].astype('Int64')
cleaned_crime_df.head(100)

,CouncilDistrict,NIBRSUCROffDesc,ReportDate,APDDistrict,LocationZip,CensusTract,XCoordinate,YCoordinate
0,4.0,Robbery,2015-01-01,E,78753,18.13,3130483.0,10102366.0
1,4.0,Robbery,2015-01-01,I,78751,21.05,3124730.0,10090296.0
2,1.0,Burglary,2015-01-01,E,78753,18.35,3135985.0,10117220.0
3,4.0,Burglary,2015-01-01,I,78753,18.13,3129896.0,10096032.0
4,2.0,Burglary,2015-01-01,F,78744,24.27,3110455.0,10039340.0
...,...,...,...,...,...,...,...,...
95,3.0,Burglary,2015-01-02,H,78741,23.18,3122189.0,10058144.0
96,10.0,Burglary,2015-01-02,B,78703,16.03,3107862.0,10082486.0
97,4.0,Burglary,2015-01-02,E,78758,18.22,3126613.0,10108839.0
98,10.0,Burglary,2015-01-02,A,78759,25.00,3109537.0,10122725.0


In [10]:
y=cleaned_crime_df['NIBRSUCROffDesc']
x=cleaned_crime_df.drop('NIBRSUCROffDesc',axis=1)

In [11]:
y.head()

0     Robbery
1     Robbery
2    Burglary
3    Burglary
4    Burglary
Name: NIBRSUCROffDesc, dtype: object

In [12]:
x.head()

,CouncilDistrict,ReportDate,APDDistrict,LocationZip,CensusTract,XCoordinate,YCoordinate
0,4.0,2015-01-01,E,78753,18.13,3130483.0,10102366.0
1,4.0,2015-01-01,I,78751,21.05,3124730.0,10090296.0
2,1.0,2015-01-01,E,78753,18.35,3135985.0,10117220.0
3,4.0,2015-01-01,I,78753,18.13,3129896.0,10096032.0
4,2.0,2015-01-01,F,78744,24.27,3110455.0,10039340.0


In [13]:
x.nunique()

CouncilDistrict       10
ReportDate          1461
APDDistrict           12
LocationZip           50
CensusTract          222
XCoordinate        32780
YCoordinate        36911
dtype: int64

In [14]:
zipcode_counts = x.LocationZip.value_counts()
zipcode_counts

78753    12403
78741    12031
78758    10397
78704    10205
78745     8912
78701     8161
78723     7975
78744     7434
78702     6588
78759     5647
78748     5315
78752     4683
78757     4350
78705     4341
78751     3493
78746     3418
78749     3103
78703     2695
78727     2370
78729     2235
78731     2082
78724     2045
78721     1924
78754     1754
78750     1627
78613     1394
78756     1351
78735     1206
78717     1201
78722     1137
78617     1036
78747      922
78726      893
78660      803
78719      673
78739      557
78736      381
78730      230
78725      199
78742      180
78653      171
78728       49
78652       47
78712       24
78732       10
78737        9
78733        5
78664        4
78734        3
78681        3
Name: LocationZip, dtype: Int64

In [15]:
x['ReportDate_Month'] = pds.DatetimeIndex(x['ReportDate']).month
x.head()

,CouncilDistrict,ReportDate,APDDistrict,LocationZip,CensusTract,XCoordinate,YCoordinate,ReportDate_Month
0,4.0,2015-01-01,E,78753,18.13,3130483.0,10102366.0,1
1,4.0,2015-01-01,I,78751,21.05,3124730.0,10090296.0,1
2,1.0,2015-01-01,E,78753,18.35,3135985.0,10117220.0,1
3,4.0,2015-01-01,I,78753,18.13,3129896.0,10096032.0,1
4,2.0,2015-01-01,F,78744,24.27,3110455.0,10039340.0,1


In [16]:
x['ReportDate_Day'] = pds.DatetimeIndex(x['ReportDate']).day
x.head()

,CouncilDistrict,ReportDate,APDDistrict,LocationZip,CensusTract,XCoordinate,YCoordinate,ReportDate_Month,ReportDate_Day
0,4.0,2015-01-01,E,78753,18.13,3130483.0,10102366.0,1,1
1,4.0,2015-01-01,I,78751,21.05,3124730.0,10090296.0,1,1
2,1.0,2015-01-01,E,78753,18.35,3135985.0,10117220.0,1,1
3,4.0,2015-01-01,I,78753,18.13,3129896.0,10096032.0,1,1
4,2.0,2015-01-01,F,78744,24.27,3110455.0,10039340.0,1,1


In [18]:
x['ReportDate'] = pds.to_datetime(x['ReportDate'])
x['ReportDate_DayOfWeek'] = x['ReportDate'].dt.dayofweek
x.head()


,CouncilDistrict,ReportDate,APDDistrict,LocationZip,CensusTract,XCoordinate,YCoordinate,ReportDate_Month,ReportDate_Day,ReportDate_DayOfWeek
0,4.0,2015-01-01,E,78753,18.13,3130483.0,10102366.0,1,1,3
1,4.0,2015-01-01,I,78751,21.05,3124730.0,10090296.0,1,1,3
2,1.0,2015-01-01,E,78753,18.35,3135985.0,10117220.0,1,1,3
3,4.0,2015-01-01,I,78753,18.13,3129896.0,10096032.0,1,1,3
4,2.0,2015-01-01,F,78744,24.27,3110455.0,10039340.0,1,1,3


In [ ]:
# Generate our categorical variable list
crime_cat = x.dtypes[x.dtypes == "object"].index.tolist()
print(crime_cat)